# Reviewing and Preparing the Data
- Proceedeeding column by column, review data and take necessary stepts to parse/clean/standardize values

In [1]:
import pandas as pd
import numpy as np
import cpi

In [2]:
# Import processed dataset
auctions = pd.read_csv(
    'data/auction_data_processed.csv', 
    usecols=[0, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14]
)

auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in
0,Pablo PICASSO,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN
1,Pablo PICASSO,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN
2,Pablo PICASSO,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN
3,Pablo PICASSO,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN
4,Pablo PICASSO,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN


## 1. `artist_name`
- Convert to title case
### 1.1 Convert to title case

In [3]:
auctions['artist_name'] = auctions['artist_name'].str.title()
auctions['artist_name'].sample(10)

22344               Peter Doig
18591                Ed Ruscha
23447             George Condo
23581         Christopher Wool
5545               Andy Warhol
19356    Pierre-Auguste Renoir
943              Pablo Picasso
23474         Christopher Wool
51475             Josef Albers
17066            Joan Mitchell
Name: artist_name, dtype: object

## 2. Merge with MoMA's `artist` Dataset for Additional Features

- Load MoMA `Artists` dataset, specifying that 0 be counted as NaN since I know that `EndDate` (i.e., the year of the artist's death) is sometimes set to 0 when not relevant (artist still living) or not known
- Standardize `auctions['artist_name']` and `Artists['DisplayName']` using `.str.lower()`
- Merge with `auctions` and `Artists` on lowercase artist name

In [4]:
# Load dataset
moma_artists = pd.read_csv('../data-exploration/data/moma/Artists.csv', usecols=[1, 3, 4, 5, 6], na_values=[0])
moma_artists.head()

,DisplayName,Nationality,Gender,BeginDate,EndDate
0,Robert Arneson,American,Male,1930.0,1992.0
1,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,Bill Arnold,American,Male,1941.0,NaN
3,Charles Arnoldi,American,Male,1946.0,NaN
4,Per Arnoldi,Danish,Male,1941.0,NaN


In [5]:
# Standardize artist names from both datasets

from unidecode import unidecode

# Ensure that strings are stripped of accents to ensure better matching
def standardize_name(name):
    return unidecode(name.lower())

moma_artists['name'] = moma_artists['DisplayName'].apply(standardize_name)
auctions['name'] = auctions['artist_name'].apply(standardize_name)

In [6]:
# Merge
auctions = pd.merge(
    left=auctions, 
    right=moma_artists,
    how='left',
    left_on='name',
    right_on='name'
)
auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,name,DisplayName,Nationality,Gender,BeginDate,EndDate
0,Pablo Picasso,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
1,Pablo Picasso,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
2,Pablo Picasso,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
3,Pablo Picasso,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
4,Pablo Picasso,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0


## 3. `StartDate` and `EndDate`

### 3.1 Confirm Dtypes

In [7]:
auctions['BeginDate'].dtype

dtype('float64')

In [8]:
auctions['EndDate'].dtype

dtype('float64')

### 3.2 Look for missings
It's expected to be missing values for `EndDate` if the artist is still alive, but if we're missing values for `StartDate` there's likely a problem. For each artist missing `StartDate` information, I'll need to manually add these.

In [9]:
auctions[auctions['BeginDate'].isna()]['artist_name'].unique()

array(['Qi Baishi', 'Fu Baoshi', 'Wu Guanzhong', 'Sanyu', 'Banksy',
       'Wassily Kandinsky', 'Ed Ruscha', 'Chu Teh-Chun', 'Pan Tianshou',
       'Francois-Xavier Lalanne', 'Kaws', 'Lin Fengmian', 'Edgar Degas',
       'Pu Ru', 'Zhou Chunya', 'Zeng Fanzhi', 'Liu Ye',
       'Tsuguharu Foujita', 'Norman Rockwell', 'Zhang Xiaogang',
       'Dong Qichang', 'Wu Hufan', 'Tamara De Lempicka', 'Wen Zhengming',
       'Kenny Scharf'], dtype=object)

In [10]:
# Specify missing data as dictionary
artist_dates = {
    'Qi Baishi': {'BeginDate':1864, 'EndDate':1957, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Fu Baoshi': {'BeginDate':1904, 'EndDate':1965, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Wu Guanzhong': {'BeginDate':1919, 'EndDate':2010, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Sanyu':{'BeginDate':1901, 'EndDate':1966, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Banksy':{'BeginDate':1974, 'Nationality':'English', 'Nationality':'English', 'Gender':'Male'},
    'Wassily Kandinsky':{'BeginDate':1866, 'EndDate':1944, 'Nationality':'Russian', 'Gender':'Male'}, 
    'Ed Ruscha':{'BeginDate':1937, 'Nationality':'American', 'Gender':'Male'}, 
    'Chu Teh-Chun':{'BeginDate':1920, 'EndDate':2014, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Pan Tianshou':{'BeginDate':1897, 'EndDate':1971, 'Nationality':'Chinese', 'Gender':'Male'},
    'Francois-Xavier Lalanne':{'BeginDate':1927, 'EndDate':2008, 'Nationality':'French', 'Gender':'Male'}, 
    'Kaws':{'BeginDate':1974, 'Nationality':'American', 'Gender':'Male'}, 
    'Lin Fengmian':{'BeginDate':1900, 'EndDate':1991, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Edgar Degas':{'BeginDate':1834, 'EndDate':1917, 'Nationality':'French', 'Gender':'Male'},
    'Pu Ru':{'BeginDate':1896, 'EndDate':1963, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Zhou Chunya':{'BeginDate':1955, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Zeng Fanzhi':{'BeginDate':1964, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Liu Ye':{'BeginDate':1964, 'Nationality':'Chinese', 'Gender':'Male'},
    'Tsuguharu Foujita':{'BeginDate':1886, 'EndDate':1968, 'Nationality':'Japanese', 'Gender':'Male'}, 
    'Norman Rockwell':{'BeginDate':1894, 'EndDate':1978, 'Nationality':'American', 'Gender':'Male'}, 
    'Zhang Xiaogang':{'BeginDate':1958, 'Nationality':'Chinese', 'Gender':'Male'},
    'Dong Qichang':{'BeginDate':1555, 'EndDate':1636, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Wu Hufan':{'BeginDate':1894, 'EndDate':1968, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Tamara De Lempicka':{'BeginDate':1898, 'EndDate':1980, 'Nationality':'Polish', 'Gender':'Female'}, 
    'Wen Zhengming':{'BeginDate':1470, 'EndDate':1559, 'Nationality':'Chinese', 'Gender':'Male'},
    'Kenny Scharf':{'BeginDate':1958, 'Nationality':'American', 'Gender':'Male'}
}

# Add missing data to DataFrame
for name, values in artist_dates.items():
    current_artist = (auctions['artist_name'] == name)

    auctions.loc[current_artist, 'BeginDate'] = values.get('BeginDate', '')
    auctions.loc[current_artist, 'EndDate'] = values.get('EndDate', np.nan)
    auctions.loc[current_artist, 'Nationality'] = values.get('Nationality', '')
    auctions.loc[current_artist, 'Gender'] = values.get('Gender', '')

Last I want to rename these columns.

In [11]:
auctions.rename(columns={'BeginDate':'birth_year', 'EndDate':'death_year'}, inplace=True)

## 4. Drop Redundant Column Post-Merge

In [12]:
auctions.drop(columns=['DisplayName', 'name'], inplace=True)
auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year
0,Pablo Picasso,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN,Spanish,Male,1881.0,1973.0
1,Pablo Picasso,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN,Spanish,Male,1881.0,1973.0
2,Pablo Picasso,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN,Spanish,Male,1881.0,1973.0
3,Pablo Picasso,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN,Spanish,Male,1881.0,1973.0
4,Pablo Picasso,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN,Spanish,Male,1881.0,1973.0


## 5. Parse `auction_date` as DateTime object

Jumping out of order for a moment since the auction date may help eliminate problematic artwork dates (i.e., an artwork completed after it was auctioned, etc.)

In [13]:
# Look for any issues
auctions['auction_date_parsed'] = pd.to_datetime(auctions['auction_date'])
auctions[['auction_date', 'auction_date_parsed']].sample(10)

,auction_date,auction_date_parsed
26685,"Nov 30, 2012",2012-11-30
27196,"Nov 13, 2002",2002-11-13
29530,"Oct 24, 2014",2014-10-24
29723,"May 11, 2012",2012-05-11
32185,"May 20, 2022",2022-05-20
45104,"Nov 15, 2012",2012-11-15
12627,"Jun 19, 2009",2009-06-19
42647,"Dec 16, 1999",1999-12-16
48868,"Mar 28, 2018",2018-03-28
8742,"Nov 13, 1991",1991-11-13


## 6. Parse `date`
Because of how I scraped this data, these values should all be prepended by `, `, so I'll start by stripping this.

### 6.1 Strip commas and spaces

In [14]:
auctions['date'] = auctions['date'].str.lstrip(', ')
auctions['date'].sample(10)

50300         1949
11792         1955
6383          1986
36990         1989
35896         1960
8708          1989
24802          NaN
14902          NaN
15926         1968
36705    1923-1994
Name: date, dtype: object

### 6.2 Examine Formats
Now let's get a sense for the various date formats and parse each one.

In [15]:
# Review `date` values that are not YYYY format
format_oneyear = (auctions['date'].str.match(r'^\d{4}$').fillna(False))

auctions.loc[~format_oneyear, 'date'].dropna().sample(10)

19488    1841-1919
2878     1928-1987
43233    1923-2002
16857    1925-1992
37120    1923-1994
17828    1881-1955
16966    1925-1992
44057    1915-1991
44633    1931-2004
44631    1931-2004
Name: date, dtype: object

In [16]:
# Review `date` values that are additionally not YYYY-YYY (or similar) format
format_span = (auctions['date'].str.match(r'^\d{4} ?[-–/]? ?\d{4}$').fillna(False))

auctions.loc[~format_oneyear & ~format_span, 'date'].dropna().sample(10)

41090         circa 1952
50034         circa 1961
25879         circa 1868
2261          circa 1979
18386    circa 1904-1906
33728        circa 1920s
13185         circa 1978
12078         circa 1966
39727         circa 1910
11600              1940s
Name: date, dtype: object

In [17]:
# Review `date` values that additionally are not prepended by 'circa'
format_circa = (auctions['date'].str.match(r'^circa ?\d{4} ?[-–/]? ?(\d{4})?$').fillna(False))

auctions.loc[~format_oneyear & ~format_span & ~format_circa, 'date'].dropna()

11600                     1940s
11614               circa 1950s
11621                     1940s
11623               1940s-1950s
14308    2005, 2005, 2005, 2005
33406                1924, 1913
33728               circa 1920s
46679                2002, 2009
50837      1936-1937, 1936-1937
Name: date, dtype: object

So it seems we're dealing with three main formats:
- A standard 4 digit year
- A spanned date consisting of two four-digit years separated by hyphens, slashes or spaces
- An approximate year or year span prepended by 'circa' and/or appended with 's'

So to parse these dates, I'm going to extract the first four-digit number as `start_date` and the last four-digit number as `end_date`, which seems like it should cover all the scenarios here. Then I'll make sure that there's no situation where `start_date > end_date`.

### 6.1 Parse

In [18]:
# Extract artwork start date
auctions['start_date'] = auctions['date'].str.extract(r'^\D*?(\d{4}).*?').astype('float')

# Extract artwork end date
auctions['end_date'] = auctions['date'].str.extract(r'^.*?(\d{4})\D*?$').astype('float')

cols = ['date', 'start_date', 'end_date']
auctions.loc[auctions['date'].notna(), cols].sample(10)

,date,start_date,end_date
11925,1969,1969.0,1969.0
10191,2007,2007.0,2007.0
18976,1961,1961.0,1961.0
3640,1962,1962.0,1962.0
23582,2006,2006.0,2006.0
13199,1969,1969.0,1969.0
46526,1991,1991.0,1991.0
42238,1968,1968.0,1968.0
5159,1964,1964.0,1964.0
7323,2005,2005.0,2005.0


So far this looks great. I just want to check the edge cases above.

In [19]:
auctions.loc[~format_oneyear & ~format_span & ~format_circa, cols].dropna()

,date,start_date,end_date
11600,1940s,1940.0,1940.0
11614,circa 1950s,1950.0,1950.0
11621,1940s,1940.0,1940.0
11623,1940s-1950s,1940.0,1950.0
14308,"2005, 2005, 2005, 2005",2005.0,2005.0
33406,"1924, 1913",1924.0,1913.0
33728,circa 1920s,1920.0,1920.0
46679,"2002, 2009",2002.0,2009.0
50837,"1936-1937, 1936-1937",1936.0,1937.0


Looks good! But actually not quite right. The value `1940s` for instance should correspond not to a start and end date of 1940 but a span of 1940-1949. Let me specifically look for dates that end in 's' and see what we're dealing with here.

In [20]:
approx_formats = (auctions['date'].str.match(r'.*?s.*?').fillna(False))

auctions.loc[approx_formats, 'date']

11600          1940s
11614    circa 1950s
11621          1940s
11623    1940s-1950s
33728    circa 1920s
Name: date, dtype: object

There's only a few of these, so I'll just correct them by hand.

In [21]:
auctions.loc[11600, ['start_date', 'end_date']] = [1940.0, 1949.0]
auctions.loc[11614, ['start_date', 'end_date']] = [1950.0, 1959.0]
auctions.loc[11621, ['start_date', 'end_date']] = [1940.0, 1949.0]
auctions.loc[11623, ['start_date', 'end_date']] = [1940.0, 1959.0]
auctions.loc[33728, ['start_date', 'end_date']] = [1920.0, 1929.0]

auctions.loc[approx_formats, cols]

,date,start_date,end_date
11600,1940s,1940.0,1949.0
11614,circa 1950s,1950.0,1959.0
11621,1940s,1940.0,1949.0
11623,1940s-1950s,1940.0,1959.0
33728,circa 1920s,1920.0,1929.0


### 6.4 Look for weirdness: Situations where `start_date > end_date`

In [22]:
weird = (auctions['start_date'] > auctions['end_date'])

auctions[weird]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
33406,Ernst Ludwig Kirchner,Sertigweg im Sommer (recto); Fehmarnküste mit ...,"1924, 1913",oil on canvas,47 ¼ x 35 5/8 in. (120.2 x 90.5 cm.),"Jun 30, 2021",Christie's,20th/21st Century: London Evening Sale,Lot48,"£982,500• US\$1,358,630","£800,000–£1,200,000(est)",NaN,German,Male,1880.0,1938.0,2021-06-30,1924.0,1913.0
41636,Francis Picabia,Nature morte à la soupière,1979-1953,oil on canvas,73.3 x 92.4 cm,"May 9, 2013","Christie's • New York, Rockefeller Center",Impressionist & Modern Day Sale,Lot238,"US\$159,750","US\$80,000–US\$120,000(est)",NaN,French,Male,1879.0,1953.0,2013-05-09,1979.0,1953.0


In [23]:
# Correct weirdness
auctions.loc[33406, ['start_date', 'end_date']] = [1913.0, 1924.0]
auctions.loc[41636, ['start_date', 'end_date']] = [1908.0, 1908.0]

auctions.loc[weird, cols]

,date,start_date,end_date
33406,"1924, 1913",1913.0,1924.0
41636,1979-1953,1908.0,1908.0


### 6.5 Look for weirdness: Situations where `end_date > death_yer` or `start_date < birth_year`

In [24]:
weird1 = (auctions['end_date'] > auctions['death_year'])
weird2 = (auctions['start_date'] < auctions['birth_year'] + 10)

print("{} records where `end_date` > `death_year`".format(len(auctions[weird1])))
print("{} records where `start_date` < `birth_year + 10`".format(len(auctions[weird2])))

58 records where `end_date` > `death_year`
6044 records where `start_date` < `birth_year + 10`


For weirdness of the first type (`end_date > death_year`, ie artwork created after artist's death), I'll just replace artwork dates with `np.nan` since this just doesn't make sense to me.

In [25]:
auctions.loc[weird1, ['start_date', 'end_date']] = np.nan

However, the second situation seems like a bigger potential issue.

In [26]:
auctions[weird2]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
136,Pablo Picasso,Tête d'homme,1881-1973,oil on canvas,61.1 x 50 cm,"Dec 2, 2020",Christie's• New York,20th Century: Hong Kong to New York,Lot43,"US\$2,670,000","US\$1,500,000–US\$2,500,000(est)",NaN,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
137,Pablo Picasso,Femme nue assise,1881-1973,oil on board laid down on panel,49.5 x 36.2 cm,"Dec 2, 2020",Christie's• New York,20th Century: Hong Kong to New York,Lot48,NaN,"US\$1,200,000–US\$1,800,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
138,Pablo Picasso,Femme debout,1881-1973,oil on canvas,129 x 96.5 cm,"Dec 2, 2020",Christie's • New York,20th Century: Hong Kong to New York,Lot33,"US\$2,550,000","US\$3,000,000–US\$5,000,000 (est)",NaN,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
140,Pablo Picasso,Nature morte aux tomates,1881-1973,oil on panel,92 x 73.5 cm,"Oct 22, 2020",Christie's • Paris,Paris Avant-garde,Lot13,"€968,000• US\$1,148,297","€900,000–€1,300,000(est)",NaN,Spanish,Male,1881.0,1973.0,2020-10-22,1881.0,1973.0
142,Pablo Picasso,La femme aux bas bleus,1881-1973,oil on board laid down on canvas,65 x 50 cm,"Oct 6, 2020",Christie's • New York,20th Century Evening Sale,Lot56,"US\$1,470,000","US\$1,000,000–US\$1,500,000 (est)",NaN,Spanish,Male,1881.0,1973.0,2020-10-06,1881.0,1973.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52971,Francis Bacon,Study of Portrait of John Edwards,1909-1992,oil on canvas,35.9 x 30.8 cm,"May 13, 2008","Christie's • New York, Rockefeller Center",Post War And Contemporary Art Evening Sale,Lot39,"US\$4,521,000","US\$4,000,000–US\$6,000,000 (est)",NaN,British,Male,1909.0,1992.0,2008-05-13,1909.0,1992.0
52972,Francis Bacon,Three Studies for Self-Portrait,1909-1992,"oil on canvas, in three parts",35.5 x 101 cm,"May 13, 2008","Christie's • New York, Rockefeller Center",Post War And Contemporary Art Evening Sale,Lot10,"US\$28,041,000","US\$25,000,000–US\$35,000,000 (est)",NaN,British,Male,1909.0,1992.0,2008-05-13,1909.0,1992.0
52974,Francis Bacon,Untitled (Head),1909-1992,oil on canvas,35.5 x 30.5cm,"Feb 6, 2008","Christie's • London, King Street",Post-War and Contemporary Art Evening Sale,Lot37,"£558,100 • US\$1,094,992","£500,000–£750,000 (est)",NaN,British,Male,1909.0,1992.0,2008-02-06,1909.0,1992.0
52981,Francis Bacon,Untitled (Figure on a Dais),1909-1992,huile sur toile,190 x 140 cm,"May 30, 2007",Christie's • Paris,Art d'Après-Guerre et Contemporain,Lot325,"€6,864,000 • US\$9,231,545","€4,000,000–€5,000,000 (est)",NaN,British,Male,1909.0,1992.0,2007-05-30,1909.0,1992.0


I can see immediately that there are some situations where the artwork date incorrectly has the artist's life span. Whenever this is the case (i.e., when `end_date == death_year and start_date == birth_year`) I'll assume the artwork dates are unknown.

In [27]:
mislabeled = (auctions['end_date'] == auctions['death_year']) & (auctions['start_date'] == auctions['birth_year'])

auctions.loc[mislabeled, ['start_date', 'end_date']] = np.nan

Now let's have another look at weirdness of the second variety.

In [28]:
auctions[(auctions['start_date'] < auctions['birth_year'] + 10)]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
342,Pablo Picasso,"Virgen de Guadalupe ,\n1848",1848,Oil on canvas,57.0 x 88.0 cm,"Sep 12, 2017",Morton Casa de Subastas • Lomas de Chapultepec,Gran Sabasta de Antiguedades,Lot138,NaN,"MX\$40,000–MX\$60,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2017-09-12,1848.0,1848.0
343,Pablo Picasso,"Divino rostro ,\n1839",1839,Oil on canvas,43.0 x 56.0 cm,"Sep 12, 2017",Morton Casa de Subastas • Lomas de Chapultepec,Gran Sabasta de Antiguedades,Lot130,NaN,"MX\$10,000–MX\$15,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2017-09-12,1839.0,1839.0
1297,Jean-Michel Basquiat,Untitled,1960,"graphite, colored pencil, oil crayon and acryl...",104.14 x 75.25 cm,"Nov 17, 2020",Sotheby's,Contemporary Art Day Auction,Lot133,"US\$1,000,100","US\$900,000–US\$1,200,000(est)",NaN,American,Male,1960.0,1988.0,2020-11-17,1960.0,1960.0
1298,Jean-Michel Basquiat,BLACK,1960,"acrylic, oil, graphite, crayon and Xerox on wood",92.7 x 126.4 x 29.2 cm,"Oct 28, 2020",Sotheby's,Contemporary Art Evening Auction,Lot18,"US\$8,134,650","US\$4,000,000–US\$6,000,000 (est)",NaN,American,Male,1960.0,1988.0,2020-10-28,1960.0,1960.0
1453,Jean-Michel Basquiat,Collaboration,1928-1987,"colored oilsticks, synthetic polymer paint and...",51.2 x 40.7 cm,"May 14, 2014","Christie's • New York, Rockefeller Center",Post-War and Contemporary Afternoon Session,Lot479,"US\$581,000","US\$300,000–US\$500,000 (est)",NaN,American,Male,1960.0,1988.0,2014-05-14,1928.0,1987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52453,Robert Indiana,SMALL DIAMOND HEXAGON 6,1928,oil on canvas,86.4 x 86.4 cm,"Oct 2, 2020",Sotheby's,Contemporary Curated,Lot41,NaN,"US\$150,000–US\$250,000(est)",1.0,American,Male,1928.0,2018.0,2020-10-02,1928.0,1928.0
52580,Robert Indiana,CLASSIC LOVE,1820,chrome-dyed hand carved tufted archival New Ze...,1820 x 1820 mm,"Sep 16, 2008","Christie's • London, South Kensington",Interiors - 20th Century Edition including Mod...,Lot172,"£1,625 • US\$2,917","£1,500–£2,000(est)",NaN,American,Male,1928.0,2018.0,2008-09-16,1820.0,1820.0
52581,Robert Indiana,CHOSEN LOVE,1820,chrome-dyed hand carved tufted archival New Ze...,1820 x 1820 mm,"Sep 16, 2008","Christie's • London, South Kensington",Interiors - 20th Century Edition including Mod...,Lot106,"£1,375 • US\$2,468","£1,000–£1,500(est)",NaN,American,Male,1928.0,2018.0,2008-09-16,1820.0,1820.0
52725,Leonora Carrington,Mi general aspirina y sus hombres,1917,oil on canvas,76.2 x 101.3 cm,"Nov 17, 2010","Christie's • New York, Rockefeller Center",Latin American Sale,Lot54,"US\$206,500","US\$150,000–US\$200,000(est)",NaN,British,Female,1917.0,2011.0,2010-11-17,1917.0,1917.0


This is much more reasonable. It's not that many records that are problematic, so I'll just assume the dates for these works are unknown and replace them with `np.nan`.

In [29]:
mislabeled = (auctions['start_date'] < auctions['birth_year'] + 10)

auctions.loc[mislabeled, ['start_date', 'end_date']] = np.nan

## 7. `dimensions`
### 7.1 Exploration
Let's see how varied this features is.

In [30]:
# Create masks for expected formats in cm, mm, or in units
expected_cm_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?cm.*?').fillna(False))
expected_in_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?in.*?').fillna(False))
expected_mm_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?mm.*?').fillna(False))

auctions.loc[~expected_cm_format & ~expected_in_format & ~expected_mm_format, 'dims'].dropna()

59                400 Diameter 390mm.
83                diameter 250mm 10in
113             width 190 mm 7 1/2 in
158                           19.5 cm
162                           19.0 cm
                     ...             
50370             45 7/8 x 34 7/8 in.
50435                    cm 40,5x32,5
51587    8⅛ by 24 in.\n20.6 by 61 cm.
51944                           6 cm.
52423                   2 by 1 5/8 in
Name: dims, Length: 723, dtype: object

In [31]:
# Ensure that each value has a unit of measurement
auctions.loc[~auctions['dims'].str.contains('in|cm|mm|CM', regex=True).fillna(False), 'dims'].dropna()

25316    dimensions variable
Name: dims, dtype: object

Because each value (except one) has a unit of measurment, I'll use that as a boundary, extracting the portion of this field specific to each unit of measurement, and then parse from there. Doing it this way, however, will remove text specifying whether a measurement is a diameter or not--that's information I want, too. For now, I'll create a flag for works that have diameter measurements, since this will require a different calculation for area.

### 7.2 Preliminary Extraction

In [32]:
# Extract cm dimensions
auctions['dims_cm'] = auctions['dims'].str.lower().str.extract(r'(\d[^(in|mm)]+ ?cm)')

# Extract in. dimensions
auctions['dims_in'] = auctions['dims'].str.lower().str.extract(r'(\d[^(cm|mm)]+ ?in)')

# Extract mm dimensions
auctions['dims_mm'] = auctions['dims'].str.lower().str.extract(r'(\d[^(in|cm)]+ ?mm)')

# Create flag for diameter
auctions['is_diameter'] = auctions['dims'].str.lower().str.match(r'.*diameter.*').fillna(False)

auctions[['dims', 'dims_cm', 'dims_in', 'dims_mm']].sample(10)

,dims,dims_cm,dims_in,dims_mm
9655,71.1 by 55.9 cm,71.1 by 55.9 cm,NaN,NaN
33031,153.6 by 128.9 cm,153.6 by 128.9 cm,NaN,NaN
36874,104.0 x 73.0 cm,104.0 x 73.0 cm,NaN,NaN
45785,60.96 x 50.8 in,NaN,60.96 x 50.8 in,NaN
45754,40.64 x 17.78 in,NaN,40.64 x 17.78 in,NaN
24737,101.6 x 68.61 in,NaN,101.6 x 68.61 in,NaN
10679,50.5 x 41.0 cm,50.5 x 41.0 cm,NaN,NaN
50194,73.3 x 60 cm.\n28 7/8 x 23 5/8 in.,73.3 x 60 cm,28 7/8 x 23 5/8 in,NaN
22618,127 x 102 cm (50 x 40 1/8 in.),127 x 102 cm,50 x 40 1/8 in,NaN
28382,120 by 100cm,120 by 100cm,NaN,NaN


This roughly looks like what's expected but let's check out the edge cases.

In [33]:
# Review extractions where `dims` is not in an expected format
edge_cases = (
    ~expected_cm_format & 
    ~expected_in_format & 
    ~expected_mm_format & 
    auctions['dims'].notna()
)

auctions.loc[edge_cases, ['dims', 'dims_cm', 'dims_in', 'dims_mm']]

,dims,dims_cm,dims_in,dims_mm
59,400 Diameter 390mm.,NaN,NaN,390mm
83,diameter 250mm 10in,NaN,10in,250mm
113,width 190 mm 7 1/2 in,NaN,7 1/2 in,190 mm
158,19.5 cm,19.5 cm,NaN,NaN
162,19.0 cm,19.0 cm,NaN,NaN
...,...,...,...,...
50370,45 7/8 x 34 7/8 in.,NaN,45 7/8 x 34 7/8 in,NaN
50435,"cm 40,5x32,5",NaN,NaN,NaN
51587,8⅛ by 24 in.\n20.6 by 61 cm.,20.6 by 61 cm,8⅛ by 24 in,NaN
51944,6 cm.,6 cm,NaN,NaN


This is looking quite good to me so far. Before continuing, I just want to ensure that there are no remaining `dims` values that were not extracted to `dims_cm`, `dims_in`, or `dims_mm`.

In [34]:
# Ensure all `dims` values were extracted in one form or another
cols = ['dims', 'dims_cm', 'dims_in', 'dims_mm']

mask = (
    auctions['dims'].notna() &
    auctions['dims_cm'].isna() &
    auctions['dims_mm'].isna() &
    auctions['dims_in'].isna()
)

auctions.loc[mask, cols]

,dims,dims_cm,dims_in,dims_mm
1335,cm 61x51,NaN,NaN,NaN
15749,"cm 41x35,5",NaN,NaN,NaN
15767,"cm 60x75,2",NaN,NaN,NaN
15797,"cm 27,5x24",NaN,NaN,NaN
15813,"cm 45,5x38",NaN,NaN,NaN
15814,"cm 24,4x31,3",NaN,NaN,NaN
16396,"cm 25,4x30,3",NaN,NaN,NaN
23191,"cm 50,5x40",NaN,NaN,NaN
25316,dimensions variable,NaN,NaN,NaN
43312,"cm 45,7x54,5",NaN,NaN,NaN


Not quite, so I'll need to extract these as well (i.e., dims prepended by 'cm')

In [35]:
auctions.loc[mask, 'dims_cm'] = (
    auctions.loc[mask, 'dims'].str.replace(',', '.')
    .str.extract(r'^cm ?(\d+\.?\d*x\d+\.?\d*)')[0]
)
auctions.loc[mask, cols]

,dims,dims_cm,dims_in,dims_mm
1335,cm 61x51,61x51,NaN,NaN
15749,"cm 41x35,5",41x35.5,NaN,NaN
15767,"cm 60x75,2",60x75.2,NaN,NaN
15797,"cm 27,5x24",27.5x24,NaN,NaN
15813,"cm 45,5x38",45.5x38,NaN,NaN
15814,"cm 24,4x31,3",24.4x31.3,NaN,NaN
16396,"cm 25,4x30,3",25.4x30.3,NaN,NaN
23191,"cm 50,5x40",50.5x40,NaN,NaN
25316,dimensions variable,NaN,NaN,NaN
43312,"cm 45,7x54,5",45.7x54.5,NaN,NaN


### 7.3 Secondary Extraction
Next, I want to extract width, height, and depth for each of these.

In [36]:
# Extract `width` dim from `dims_cm`
auctions['width_cm'] = auctions['dims_cm'].str.extract(r'^(\d+.*?)[^\d\. ].*?$')

# Extract `height` dim from `dims_cm`
auctions['height_cm'] = auctions['dims_cm'].str.extract(r'^\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

# Extract `depth` dim from `dims_cm`
auctions['depth_cm'] = auctions['dims_cm'].str.extract(r'^\d+.*?[^\d\.,].*?\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

auctions.loc[auctions['dims_cm'].notna(), ['dims_cm', 'width_cm', 'height_cm', 'depth_cm']].sample(20)

,dims_cm,width_cm,height_cm,depth_cm
6737,195 by 130 cm,195,130,NaN
2310,101.6 x 101.6cm,101.6,101.6,NaN
5913,50 x 61 cm,50,61,NaN
29022,76.2 x 101.6 cm,76.2,101.6,NaN
26280,46.3 x 56.5 cm,46.3,56.5,NaN
29133,183.3 x 274.2 cm,183.3,274.2,NaN
46490,190 by 285 cm,190,285,NaN
25625,152.4 x 152.4 cm,152.4,152.4,NaN
4568,102 x 102.2cm,102,102.2,NaN
4460,103 x 98 x 31.8cm,103,98,31.8


This looks good to me, so I'll do the same for the other units of measurement.

In [37]:
# Extract `width` dim from `dims_mm`
auctions['width_mm'] = auctions['dims_mm'].str.extract(r'^(\d+.*?)[^\d\. ].*?$')

# Extract `height` dim from `dims_mm`
auctions['height_mm'] = auctions['dims_mm'].str.extract(r'^\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

# Extract `depth` dim from `dims_mm`
auctions['depth_mm'] = auctions['dims_mm'].str.extract(r'^\d+.*?[^\d\.,].*?\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

auctions.loc[auctions['dims_mm'].notna(), ['dims_mm', 'width_mm', 'height_mm', 'depth_mm']].sample(20)

,dims_mm,width_mm,height_mm,depth_mm
126,165 by 165 by 21 mm,165,165,21
133,215 by 215 by 20 mm,215,215,20
252,260mm,260,NaN,NaN
80,170 by 170mm,170,170,NaN
16261,305 x 305 x 40 mm,305,305,40
118,165 by 165 by 21 mm,165,165,21
39385,1200 x 1200 x 50 mm,1200,1200,50
81,314 by 315mm,314,315,NaN
128,215 by 212 by 20 mm,215,212,20
254,160mm,160,NaN,NaN


Inches is a little trickier since I've noticed some numbers use fractions, so I have to account for this.

In [38]:
# Extract `width` dim from `dims_in`
auctions['width_in'] = auctions['dims_in'].str.extract(r'^(\d+.*?)[^\d\. /].*?$')

# Extract `height` dim from `dims_in`
auctions['height_in'] = auctions['dims_in'].str.extract(r'^\d+.*?[^\d\., /].*?(\d+.*?)[^\d\., /].*?$')

# Extract `depth` dim from `dims_in`
auctions['depth_in'] = auctions['dims_in'].str.extract(r'^\d+.*?[^\d\., /].*?\d+.*?[^\d\., /].*?(\d+.*?)[^\d\., /].*?$')

auctions.loc[auctions['dims_in'].notna(), ['dims_in', 'width_in', 'height_in', 'depth_in']].sample(20)

,dims_in,width_in,height_in,depth_in
30084,50.8 x 58.42 in,50.8,58.42,NaN
51853,30.51 x 23.01 in,30.51,23.01,NaN
42598,25.1 x 19.38 in,25.1,19.38,NaN
33561,48.26 x 73.66 in,48.26,73.66,NaN
37385,73.66 x 72.39 in,73.66,72.39,NaN
25985,33.02 x 40.64 in,33.02,40.64,NaN
16565,178.99 x 192.0 in,178.99,192.0,NaN
35532,100.03 x 80.98 in,100.03,80.98,NaN
22182,132.08 x 96.52 in,132.08,96.52,NaN
43988,72 x 42 in,72,42,NaN


In [39]:
# Create function to parse fractions
import re

def parse_measurement(measurement):
      
    match = re.match(r'(\d+) (\d+)\/(\d+)', measurement)
    
    if match:
        num = int(match.group(1)) + int(match.group(2)) / int(match.group(3))
    else:
        num = float(re.match(r'(\d+(\.\d+)?)', measurement).group(1))
    
    return num

In [40]:
# Create masks
has_width_in = auctions['width_in'].notna()
has_height_in = auctions['height_in'].notna()
has_depth_in = auctions['depth_in'].notna()

# Parse fractions
auctions.loc[has_width_in, 'width_in'] = auctions.loc[has_width_in, 'width_in'].apply(parse_measurement)
auctions.loc[has_height_in, 'height_in'] = auctions.loc[has_height_in, 'height_in'].apply(parse_measurement)
auctions.loc[auctions['depth_in'].notna(), 'depth_in'] = auctions.loc[auctions['depth_in'].notna(), 'depth_in'].apply(parse_measurement)

cols = ['dims_in', 'width_in', 'height_in', 'depth_in']

auctions.loc[auctions['dims_in'].str.contains('/').fillna(False), cols]

,dims_in,width_in,height_in,depth_in
6,8 3/4 x 10 1/2 in,8.75,10.5,NaN
7,15 x 21 3/4 in,15.0,21.75,NaN
8,36 3/8 x 14 3/8 in,36.375,14.375,NaN
33,7 1/4 x 9 3/8 in,7.25,9.375,NaN
66,4 ¾ x 7 1/8 in,4.0,7.125,NaN
...,...,...,...,...
52433,24 x 24 1/8 in,24.0,24.125,NaN
52816,82 1/2 x 58 1/8 in,82.5,58.125,NaN
52877,25 3/4 x 21 7/8 in,25.75,21.875,NaN
52879,57 1/4 x 43 in,57.25,43.0,NaN


### 7.4 Convert Measurment Features to Float

In [41]:
# cm
print("`width_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_cm'].dropna(), errors='coerce').isna().sum()))
print("`height_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_cm'].dropna(), errors='coerce').isna().sum()))
print("`depth_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_cm'].dropna(), errors='coerce').isna().sum()))

# in
print("`width_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_in'].dropna(), errors='coerce').isna().sum()))
print("`height_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_in'].dropna(), errors='coerce').isna().sum()))
print("`depth_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_in'].dropna(), errors='coerce').isna().sum()))

# mm
print("`width_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_mm'].dropna(), errors='coerce').isna().sum()))
print("`height_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_mm'].dropna(), errors='coerce').isna().sum()))
print("`depth_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_mm'].dropna(), errors='coerce').isna().sum()))

`width_cm`: 5 non-numeric values.
`height_cm`: 16 non-numeric values.
`depth_cm`: 0 non-numeric values.
`width_in`: 0 non-numeric values.
`height_in`: 0 non-numeric values.
`depth_in`: 0 non-numeric values.
`width_mm`: 0 non-numeric values.
`height_mm`: 0 non-numeric values.
`depth_mm`: 0 non-numeric values.


I'll coerce the non-numeric `cm` values to NaN and then conver all features to floats.

In [42]:
auctions['width_cm'] = pd.to_numeric(auctions['width_cm'], errors='coerce').astype('float')
auctions['height_cm'] = pd.to_numeric(auctions['height_cm'], errors='coerce').astype('float')
auctions['depth_cm'] = auctions['depth_cm'].astype('float')

auctions['width_in'] = auctions['width_in'].astype('float')
auctions['height_in'] = auctions['height_in'].astype('float')
auctions['depth_in'] = auctions['depth_in'].astype('float')

auctions['width_mm'] = auctions['width_mm'].astype('float')
auctions['height_mm'] = auctions['height_mm'].astype('float')
auctions['depth_mm'] = auctions['depth_mm'].astype('float')

### 7.5 Standardizing Units of Measurement to cm
Last let's see where we need to convert either mm or in to cm (scenarios where `dims_cm` is null but either `dims_in` or `dims_mm` is not).

First, the easy task: converting mm to cm

In [43]:
# Review situations where we have mm but not cm
cols = ['dims', 'dims_cm', 'dims_mm', 'dims_in']

have_cm_dims = (auctions['dims_cm'].notna())
have_mm_dims = (auctions['dims_mm'].notna())
have_in_dims = (auctions['dims_in'].notna())

auctions.loc[~have_cm_dims & have_mm_dims, cols]

,dims,dims_cm,dims_mm,dims_in
22,185 by 184mm\n7¼ by 7¼in,NaN,185 by 184mm,7¼ by 7¼in
23,190 by 189mm\n7½ by 7½in,NaN,190 by 189mm,7½ by 7½in
59,400 Diameter 390mm.,NaN,390mm,NaN
79,314 by 315mm\n12⅜ by 12 3/8in,NaN,314 by 315mm,12⅜ by 12 3/8in
80,170 by 170mm\n6¾ by 6¾in,NaN,170 by 170mm,6¾ by 6¾in
81,314 by 315mm\n12⅜ by 12 3/8in,NaN,314 by 315mm,12⅜ by 12 3/8in
82,310 by 310mm\n12⅛ by 12 1/8in,NaN,310 by 310mm,12⅛ by 12 1/8in
83,diameter 250mm 10in,NaN,250mm,10in
113,width 190 mm 7 1/2 in,NaN,190 mm,7 1/2 in
114,310 by 310 by 20 mm\n12 by 12 by 3/4 in\n515 b...,NaN,310 by 310 by 20 mm,12 by 12 by 3/4 in


In [44]:
# Convert mm to cm when we have mm but not cm
mm_dims = ['width_mm', 'height_mm', 'depth_mm']
cm_dims = ['width_cm', 'height_cm', 'depth_cm']

for dims in zip(cm_dims, mm_dims):
    auctions.loc[~have_cm_dims & have_mm_dims, dims[0]] = auctions.loc[~have_cm_dims & have_mm_dims, dims[1]]/ 10

auctions.loc[~have_cm_dims & have_mm_dims, cm_dims + mm_dims]

,width_cm,height_cm,depth_cm,width_mm,height_mm,depth_mm
22,18.5,18.4,NaN,185.0,184.0,NaN
23,19.0,18.9,NaN,190.0,189.0,NaN
59,39.0,NaN,NaN,390.0,NaN,NaN
79,31.4,31.5,NaN,314.0,315.0,NaN
80,17.0,17.0,NaN,170.0,170.0,NaN
81,31.4,31.5,NaN,314.0,315.0,NaN
82,31.0,31.0,NaN,310.0,310.0,NaN
83,25.0,NaN,NaN,250.0,NaN,NaN
113,19.0,NaN,NaN,190.0,NaN,NaN
114,31.0,31.0,2.0,310.0,310.0,20.0


Now we can convert inches dims to cm dims where we're still missing the latter.

In [45]:
# Review situations where we have in. but not cm or mm
cols = ['dims', 'dims_cm', 'dims_mm', 'dims_in']

have_cm_dims = (auctions['dims_cm'].notna())
have_mm_dims = (auctions['dims_mm'].notna())
have_in_dims = (auctions['dims_in'].notna())

auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, cols]

,dims,dims_cm,dims_mm,dims_in
24,53⁄4 x 57⁄8 in. (14.5 x 14.8 in.),NaN,NaN,53⁄4 x 57⁄8 in
270,19 5/8 x 28 5/8 in.,NaN,NaN,19 5/8 x 28 5/8 in
274,15 by 15 in,NaN,NaN,15 by 15 in
416,49.83 x 65.07 in,NaN,NaN,49.83 x 65.07 in
417,19.0 x 14.3 in,NaN,NaN,19.0 x 14.3 in
...,...,...,...,...
53029,144.78 x 198.12 in,NaN,NaN,144.78 x 198.12 in
53030,68.58 x 83.82 in,NaN,NaN,68.58 x 83.82 in
53031,147.32 x 198.12 in,NaN,NaN,147.32 x 198.12 in
53032,116.84 x 152.4 in,NaN,NaN,116.84 x 152.4 in


In [46]:
# Convert in. to cm when we have in. but not cm or mm
in_dims = ['width_in', 'height_in', 'depth_in']
cm_dims = ['width_cm', 'height_cm', 'depth_cm']

for dims in zip(cm_dims, in_dims):
    auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, dims[0]] = auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, dims[1]] * 2.54

auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, cm_dims + in_dims]

,width_cm,height_cm,depth_cm,width_in,height_in,depth_in
24,134.6200,10.1600,144.78,53.000,4.000,57.0
270,49.8475,72.7075,NaN,19.625,28.625,NaN
274,38.1000,38.1000,NaN,15.000,15.000,NaN
416,126.5682,165.2778,NaN,49.830,65.070,NaN
417,48.2600,36.3220,NaN,19.000,14.300,NaN
...,...,...,...,...,...,...
53029,367.7412,503.2248,NaN,144.780,198.120,NaN
53030,174.1932,212.9028,NaN,68.580,83.820,NaN
53031,374.1928,503.2248,NaN,147.320,198.120,NaN
53032,296.7736,387.0960,NaN,116.840,152.400,NaN


## 8. `title`, `medium`, `auction_sale`, `auction_lot`
This I'm going to ignore since I don't see it being a meaningful feature in my model. Perhaps down the road I can incorporate NLP to make use of features like `title`, `medium`, and `auction_sale`

## 9 `auction_house`

In [47]:
# Review `auction_house` feature
auctions.loc[auctions['auction_house'].str.contains('•'), 'auction_house'].value_counts()

Christie's • New York                           7218
Christie's • London                             3471
Christie's • New York, Rockefeller Center       3026
Christie's • London, King Street                1910
Christie's• New York                            1424
                                                ... 
Matsa• Ramat Gan                                   1
Wolf's • Cleveland OH                              1
Charlton Hall Galleries • Columbia SC              1
Museumsbygningen Kunstauktioner • Copenhagen       1
Selkirk's • Saint Louis                            1
Name: auction_house, Length: 1313, dtype: int64

In [48]:
# Extract location information where available
auctions['auction_house_loc'] = auctions['auction_house'].str.extract(r'• ?([^,]*)')

auctions.loc[auctions['auction_house_loc'].notna(), ['auction_house', 'auction_house_loc']].sample(10)

,auction_house,auction_house_loc
52345,Finarte Semenzato • Milan,Milan
5210,Ketterer Kunst • Munich,Munich
46947,Grisebach • Berlin,Berlin
35167,Matsart Gallery • Tel Aviv,Tel Aviv
27643,Christie's • Hong Kong,Hong Kong
24171,Christie's• Hong Kong,Hong Kong
34662,"Christie's • New York, Rockefeller Center",New York
51293,"Christie's • New York, Rockefeller Center",New York
41180,Christie's • London,London
32444,Christie's • New York,New York


In [49]:
auctions['auction_house_loc'].value_counts().head(20)

New York                14639
London                   8308
Paris                    4713
Hong Kong                2149
Cologne                   644
Munich                    483
Beijing                   472
Poulain                   403
Zurich                    400
Berlin                    392
Milan                     372
Vienna                    367
Bern                      345
Tokyo                     311
South Kensington          271
Royere et Lajeunesse      262
Prato                     254
Amsterdam                 247
Taipei                    246
East                      238
Name: auction_house_loc, dtype: int64

In [50]:
# Extract auction house name information
auctions['auction_house_name'] = auctions['auction_house'].str.extract(r'^([^•]*)')[0].str.strip()

auctions.loc[auctions['auction_house_loc'].isna(), ['auction_house', 'auction_house_name']].sample(10)

,auction_house,auction_house_name
37359,Sotheby's,Sotheby's
17171,Christie's,Christie's
34134,Christie's,Christie's
36787,Sotheby's,Sotheby's
34464,Sotheby's,Sotheby's
33065,Sotheby's,Sotheby's
3408,Christie's,Christie's
37289,Phillips,Phillips
26072,Sotheby's,Sotheby's
2548,Sotheby's,Sotheby's


In [51]:
# Standardize `phillips` auction house name
phillips = (auctions['auction_house_name'].str.startswith('Phillips'))

auctions.loc[phillips, 'auction_house_name'] = 'Phillips'

auctions['auction_house_name'].value_counts().head(20)

Christie's                      24605
Sotheby's                        9230
Phillips                         3935
Artcurial                         781
Bonhams                           773
Cornette de Saint Cyr             504
Ketterer Kunst                    462
Tajan                             405
Kunsthaus Lempertz                385
Dorotheum                         374
Poly International Auction        363
Ravenel                           334
Galerie Kornfeld                  310
Guy Loudmer                       300
Francis Briest                    276
Villa Grisebach Auktionen         269
Koller Auktionen                  265
Versailles Encheres - Perrin      262
Farsetti Arte                     250
China Guardian                    215
Name: auction_house_name, dtype: int64

## 10. `price_realized`

### 10.1 Extract Price

In [52]:
# Extract USD price from `price_realized`
auctions['price_realized_USD'] = auctions['price_realized'].str.replace(',','').str.extract(r'US\\\$(\d+)').astype('float')

auctions[['price_realized', 'price_realized_USD']].sample(20)

,price_realized,price_realized_USD
8657,NaN,NaN
35085,"US\$74,500",74500.0
4880,NaN,NaN
21992,NaN,NaN
17368,"US\$600,000",600000.0
11728,"HK\$16,000,000 • US\$2,056,973",2056973.0
30643,"€187,500 • US\$224,136",224136.0
28522,NaN,NaN
3546,NaN,NaN
32482,NaN,NaN


### 10.2 Create feature for adjusted price in constant 2022 dollars
For this I'll use the `cpi` library. It currently only supports years up to 2022, so I'll leave 2023 auction values as is, and everthing else I'll inflate.

In [53]:
# Create `auction_year` feature
auctions['auction_year'] = auctions['auction_date_parsed'].dt.year

# Create mask for valid pre-2023 auctions
pre2023 = (
    (auctions['auction_year'] < 2023) & 
    (auctions['auction_year'].notna()) & 
    (auctions['price_realized_USD'].notna())
)

# Inflate pre-2023 values to 2022 prices
auctions.loc[pre2023, 'price_realized_USD_constant_2022'] = (
    auctions.loc[pre2023, :].apply(lambda x: cpi.inflate(x.price_realized_USD, x.auction_year), axis=1)
)

# Keep 2023 values as is
auctions.loc[~pre2023, 'price_realized_USD_constant_2022'] = auctions.loc[~pre2023, 'price_realized_USD']

# Preview results
auctions[['auction_year', 'price_realized_USD', 'price_realized_USD_constant_2022']].sample(10)

,auction_year,price_realized_USD,price_realized_USD_constant_2022
35680,2000,NaN,NaN
52014,2006,85000.0,123391.245040
50637,1998,NaN,NaN
40087,2009,NaN,NaN
31745,2009,306618.0,418264.871747
17070,1988,130000.0,321598.901099
34290,2018,136897.0,159547.888092
22731,2019,740000.0,847090.828728
18816,2008,11250.0,15291.792265
19706,2013,395000.0,496223.444670


## 11. `estimate_range`
I'm going to ignore this for the time being since the idea would be to make a prediction without the benefit of the estimate.

## 12. Engineer Features
- artwork area (cm sq): `width_cm` x `height_cm` or pi x (`width_cm` / 2) ^ 2 for circular works
- artwork volume (cm cu): `width_cm` x `height_cm` x `depth_cm` (shouldn't come up since we're dealing with 2D paintings, but there are a number of works with depth measurements regardless)
- living: was the artist living at the time of auction?
- years after death: years after artist death that the work was auctioned (if artist was deceased)
- age_at_auction: artist age at time of auction (if artist was alive)
- age_at_artwork_completion: artist age when artwork completed (proxy for early-, mid-, and late-career)
- artwork age: artwork age at time of auction
- years ago: years before 2023 that auction was held

### 12.1 `area_cm_sq` (no diameter)

In [54]:
# Compute area for standard works (no diameter measurement)
has_diameter = (auctions['is_diameter'] == 1)

auctions.loc[~has_diameter, 'area_cm_sq'] = (
    auctions.loc[~has_diameter, 'width_cm'] * auctions.loc[~has_diameter, 'height_cm']
)

cols = ['dims_cm', 'is_diameter', 'width_cm', 'height_cm', 'depth_cm', 'area_cm_sq']

auctions[cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq
41175,73 x 54 cm,False,73.000,54.0000,NaN,3942.000000
26695,NaN,False,193.548,254.8382,NaN,49323.423934
49425,14.0 x 22.9 cm,False,14.000,22.9000,NaN,320.600000
10448,68.6 x 101.0 cm,False,68.600,101.0000,NaN,6928.600000
50493,75.8 x 95.9 cm,False,75.800,95.9000,NaN,7269.220000
25562,NaN,False,129.540,120.6500,NaN,15629.001000
33825,NaN,False,NaN,NaN,NaN,NaN
38376,25.4 x 35.6 cm,False,25.400,35.6000,NaN,904.240000
22356,302.8 x 200.0 cm,False,302.800,200.0000,NaN,60560.000000
48453,243.5 x 133.4 cm,False,243.500,133.4000,NaN,32482.900000


### 12.2 `area_cm_sq`  (with diamter)

In [55]:
# Compute area for non-standard works (with diameter measurement)
auctions.loc[has_diameter, 'area_cm_sq'] = (
    3.1415926 * (auctions.loc[has_diameter, 'width_cm'] / 2) ** 2
)

auctions.loc[has_diameter, cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq
47972,50.8 cm,True,50.8,NaN,NaN,2026.829882
48067,177.8 cm,True,177.8,NaN,NaN,24828.666052
39159,40 cm,True,40.0,NaN,NaN,1256.637040
29276,223.5 cm,True,223.5,NaN,NaN,39232.404738
25300,91.4 cm,True,91.4,NaN,NaN,6561.184729
248,NaN,True,25.2,NaN,NaN,498.759241
227,NaN,True,31.5,NaN,NaN,779.311314
31758,290.2 cm,True,290.2,NaN,NaN,66143.122016
23871,152.4 cm,True,152.4,NaN,NaN,18241.468936
252,NaN,True,26.0,NaN,NaN,530.929149


### 12.3 `volume_cm_cu` 

In [56]:
# Compute volume for works with depth measurement
auctions['volume_cm_cu'] = (
    auctions['width_cm'] * auctions['height_cm'] * auctions['depth_cm']
)

cols = ['dims_cm', 'is_diameter', 'width_cm', 'height_cm', 'depth_cm', 'area_cm_sq', 'volume_cm_cu']

auctions.loc[auctions['depth_cm'].notna(), cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq,volume_cm_cu
31769,166.5 x 70 x 53 cm,False,166.5000,70.0000,53.0000,11655.000000,6.177150e+05
9695,50 x 91 x 17cm,False,50.0000,91.0000,17.0000,4550.000000,7.735000e+04
116,NaN,False,16.5000,16.0000,2.0000,264.000000,5.280000e+02
40914,NaN,False,113.8174,130.8354,2.5146,14891.345056,3.744578e+04
14636,NaN,False,40.0050,67.1576,36.7030,2686.639788,9.860774e+04
46777,38.0 x 38.0 x 8.9 cm,False,38.0000,38.0000,8.9000,1444.000000,1.285160e+04
23831,80 x 20.5 x 4 cm,False,80.0000,20.5000,4.0000,1640.000000,6.560000e+03
46755,169.8 x 249.5 x 7cm,False,169.8000,249.5000,7.0000,42365.100000,2.965557e+05
28983,485.09 x 243.21 x 10.21 cm,False,485.0900,243.2100,10.2100,117978.738900,1.204563e+06
4128,NaN,False,127.0254,127.0254,6.3246,16135.452245,1.020503e+05


### 12.4 `living`

In [57]:
# Create mask for if artist is still living or was living at time of auction
mask = (
    (auctions['auction_date_parsed'].dt.year <= auctions['death_year']) # Living at time of auction
    | auctions['death_year'].isna() # Still living
)

auctions['living'] = np.where(mask, 1, 0)

cols = ['auction_date_parsed', 'death_year', 'living']

auctions[cols].sample(20)

,auction_date_parsed,death_year,living
47444,2018-11-15,2017.0,0
23393,2001-11-16,NaN,1
22860,2017-12-13,NaN,1
17985,1998-12-09,1955.0,0
50798,2016-11-17,1946.0,0
50099,2014-05-13,2019.0,1
44683,2007-05-17,2004.0,0
35136,2009-12-08,1999.0,0
38413,2016-10-09,2011.0,0
49323,2002-12-04,1940.0,0


### 12.5 `years_after_death_of_auction`

In [58]:
# Compute years after artist's death that auction was held for artists no longer living at time of auction
deceased = (auctions['living'] == 0)

auctions.loc[deceased, 'years_after_death_of_auction'] = (
    auctions.loc[deceased, 'auction_date_parsed'].dt.year - auctions.loc[deceased, 'death_year']
)

cols = ['auction_date_parsed', 'death_year', 'years_after_death_of_auction']

auctions[cols].sample(10)

,auction_date_parsed,death_year,years_after_death_of_auction
10073,2013-05-26,NaN,NaN
34322,2018-05-16,1999.0,19.0
32960,2014-09-23,NaN,NaN
45821,1998-05-12,1963.0,35.0
36902,2016-10-27,1994.0,22.0
23859,2020-11-17,NaN,NaN
11458,1996-12-10,2011.0,NaN
10232,2009-10-26,NaN,NaN
9837,2016-05-29,NaN,NaN
26141,2015-11-05,1903.0,112.0


### 12.6 `artist_age_at_auction`

In [59]:
# Compute artist's age at auction if artist living at time of auction

auctions.loc[~deceased, 'artist_age_at_auction'] = (
    auctions.loc[~deceased, 'auction_date_parsed'].dt.year - auctions.loc[~deceased, 'birth_year']
)

cols = ['auction_date_parsed', 'birth_year', 'death_year', 'artist_age_at_auction']

auctions[cols].sample(10)

,auction_date_parsed,birth_year,death_year,artist_age_at_auction
45280,2011-09-22,1969.0,NaN,42.0
51232,2008-05-14,1888.0,1976.0,NaN
18665,2016-02-10,1937.0,NaN,79.0
3596,2011-11-07,1928.0,1987.0,NaN
37194,2012-11-28,1923.0,1994.0,NaN
26300,2009-05-06,1830.0,1903.0,NaN
39151,2018-05-27,1962.0,NaN,56.0
2347,2019-11-14,1928.0,1987.0,NaN
40445,1988-07-29,1867.0,1947.0,NaN
38545,2002-11-14,1928.0,2011.0,74.0


### 12.7 `artist_age_at_artwork_completeion`

In [60]:
# Compute artist age at time of artwork completion
auctions['artist_age_at_artwork_completion'] = auctions['end_date'] - auctions['birth_year']

cols = ['end_date', 'birth_year', 'artist_age_at_artwork_completion']

auctions[cols].sample(10)

,end_date,birth_year,artist_age_at_artwork_completion
27738,2014.0,1949.0,65.0
1569,NaN,1960.0,NaN
14590,NaN,1959.0,NaN
51312,NaN,1888.0,NaN
36017,1954.0,1928.0,26.0
37189,1988.0,1923.0,65.0
34986,1976.0,1928.0,48.0
22934,1983.0,1957.0,26.0
2169,1985.0,1928.0,57.0
24673,1963.0,1919.0,44.0


### 12.8 `artwork_age_at_auction`

In [61]:
# Compute artwork age at time of auction
auctions['artwork_age_at_auction'] = auctions['auction_date_parsed'].dt.year - auctions['end_date']

cols = ['auction_date_parsed', 'end_date', 'artwork_age_at_auction']

auctions[cols].sample(10)

,auction_date_parsed,end_date,artwork_age_at_auction
4908,2004-05-12,1986.0,18.0
22183,2005-11-25,1995.0,10.0
2602,2017-06-27,1975.0,42.0
52008,2007-02-09,1991.0,16.0
52876,2022-05-19,1971.0,51.0
48344,1999-05-20,1984.0,15.0
18286,2007-03-30,1930.0,77.0
32458,2011-05-26,1977.0,34.0
48334,2001-05-12,1988.0,13.0
1800,1998-11-12,1982.0,16.0


### 12.8 `years_ago_of_auction`

In [62]:
# Compute how many years from 2023 auction was held
auctions['years_ago_of_auction'] = 2023 - auctions['auction_date_parsed'].dt.year

cols = ['auction_date_parsed', 'years_ago_of_auction']

auctions[cols].sample(10)

,auction_date_parsed,years_ago_of_auction
22655,2021-04-07,2
47712,2020-07-03,3
30534,2011-10-03,12
9481,2020-06-19,3
22476,2009-10-17,14
19262,2019-05-14,4
1406,2016-11-10,7
28785,2021-11-18,2
45558,2014-11-06,9
48220,2010-03-11,13


## 13 Re-order columns and Export CSV

In [63]:
new_col_order = [
    'artist_name', 'Nationality', 'Gender', 'birth_year', 'death_year',
    'title', 'date', 'start_date', 'end_date', 'medium', 
    'dims', 'dims_cm', 'dims_in', 'dims_mm', 'is_diameter', 
    'width_cm', 'height_cm', 'depth_cm', 
    'width_mm', 'height_mm', 'depth_mm', 
    'width_in', 'height_in', 'depth_in', 
    'area_cm_sq', 'volume_cm_cu', 
    'auction_date', 'auction_date_parsed', 'auction_house', 'auction_house_loc', 'auction_house_name', 'auction_sale', 'auction_lot', 'estimate', 
    'price_realized','price_realized_USD','bought_in',
    'living', 'years_after_death_of_auction', 'artist_age_at_auction',
    'artist_age_at_artwork_completion', 'artwork_age_at_auction','years_ago_of_auction'
]

auctions[new_col_order]

,artist_name,Nationality,Gender,birth_year,death_year,title,date,start_date,end_date,medium,...,estimate,price_realized,price_realized_USD,bought_in,living,years_after_death_of_auction,artist_age_at_auction,artist_age_at_artwork_completion,artwork_age_at_auction,years_ago_of_auction
0,Pablo Picasso,Spanish,Male,1881.0,1973.0,Fillette au bateau (Maya),1938,1938.0,1938.0,oil on canvas,...,NaN,NaN,NaN,NaN,0,50.0,NaN,57.0,85.0,0
1,Pablo Picasso,Spanish,Male,1881.0,1973.0,Homme assis,1969,1969.0,1969.0,oil on panel laid down on cradled panel,...,"HK\$8,200,000–HK\$12,800,000(est)","HK\$10,650,000• US\$1,363,489",1363489.0,NaN,0,49.0,NaN,88.0,53.0,1
2,Pablo Picasso,Spanish,Male,1881.0,1973.0,Buffalo Bill,1911,1911.0,1911.0,oil and sand on canvas,...,"US\$10,000,000–US\$15,000,000(est)","US\$12,412,500",12412500.0,NaN,0,49.0,NaN,30.0,111.0,1
3,Pablo Picasso,Spanish,Male,1881.0,1973.0,Homme à la moustache,1970,1970.0,1970.0,oil and oil stick on panel,...,"US\$4,000,000–US\$6,000,000(est)","US\$4,620,000",4620000.0,NaN,0,49.0,NaN,89.0,52.0,1
4,Pablo Picasso,Spanish,Male,1881.0,1973.0,Le peintre et son modèle,1964,1964.0,1964.0,oil and ripolin on canvas,...,"US\$8,000,000–US\$12,000,000(est)","US\$10,351,500",10351500.0,NaN,0,49.0,NaN,83.0,58.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53029,Francis Bacon,British,Male,1909.0,1992.0,Figure with 2 owls - Study after Velazquez Inn...,NaN,NaN,NaN,"oil, sand on canvas",...,NaN,"US\$1,200,000",1200000.0,NaN,1,NaN,78.0,NaN,NaN,36
53030,Francis Bacon,British,Male,1909.0,1992.0,Study for a portrait,1966,1966.0,1966.0,oil on canvas,...,NaN,"£380,000 • US\$611,800",611800.0,NaN,1,NaN,78.0,57.0,21.0,36
53031,Francis Bacon,British,Male,1909.0,1992.0,Portrait of George Dyer talking,1966,1966.0,1966.0,oil on canvas,...,NaN,"US\$1,300,000",1300000.0,NaN,1,NaN,78.0,57.0,21.0,36
53032,Francis Bacon,British,Male,1909.0,1992.0,Study for Portrait II,NaN,NaN,NaN,oil on canvas,...,NaN,"US\$1,600,000",1600000.0,NaN,1,NaN,78.0,NaN,NaN,36


In [64]:
auctions.to_csv('data/auction_data_cleaned.csv')